# Kinolights 웹크롤링

In [1]:
!pip install user_agent

In [2]:
import requests
from bs4 import BeautifulSoup
from user_agent import generate_user_agent

header = {'User-Agent': generate_user_agent()}

# 크롤링할 웹 페이지의 URL
url = "https://m.kinolights.com/ranking/netflix"

# 웹 페이지 내용을 가져옴
response = requests.get(url, headers=header)

# HTTP 응답 상태 코드 확인 (200이면 성공)
try:
    response.raise_for_status()  # Raises an HTTPError if the response status is not 200 OK
    # BeautifulSoup을 사용하여 HTML을 파싱
    soup = BeautifulSoup(response.text, 'html.parser')

    # 원하는 데이터 추출
    # 예를 들어, 웹 페이지의 모든 링크를 가져오려면:
    links = soup.find_all('a')

    # 추출한 데이터 출력
    for link in links:
        print(link.get('href'))
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP 요청 실패: {http_err}")
except Exception as e:
    print(f"오류 발생: {e}")


/search
/
/search
/community/feed
/user/greetings


- 404 에러 발생 시, 아래 코드 처럼 user_agent를 추가하기

```python
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from user_agent import generate_user_agent

header = {'User-Agent': generate_user_agent()}
url = "https://m.kinolights.com/ranking/netflix"
req = Request(url, headers=header)
res = urlopen(req)
soup = BeautifulSoup(res.read(), 'html.parser')
```

In [3]:
soup

<!DOCTYPE html>

<html data-n-head-ssr="">
<head>
<title>넷플릭스 인기 랭킹 차트 TOP 20 - 키노라이츠</title><meta charset="utf-8" data-n-head="ssr"/><meta content="width=device-width, initial-scale=1, user-scalable=no" data-n-head="ssr" name="viewport"/><meta content="yes" data-n-head="ssr" name="mobile-web-app-capable"/><meta content="yes" data-n-head="ssr" name="apple-mobile-web-app-capable"/><meta content="키노라이츠 – 영화 드라마 보기 전 필수 앱" data-n-head="ssr" name="application_name"/><meta content="video.movie" data-hid="ogtype" data-n-head="ssr" property="og:type"/><meta content="키노라이츠" data-hid="author" data-n-head="ssr" name="author"/><meta content="키노라이츠 – 영화 드라마 보기 전 필수 앱" data-hid="appname" data-n-head="ssr" name="application_name"/><meta content="넷플릭스 인기 랭킹 차트 TOP 20 - 키노라이츠" data-hid="ogtitle" data-n-head="ssr" name="og:title"/><meta content="오늘의 넷플릭스 인기 순위를 키노라이츠에서 확인해보세요" data-hid="ogdesc" data-n-head="ssr" name="og:description"/><meta content="https://static.kinolights.com/seo/ranking_netflix.png

In [4]:
data = []

# ranking-list-wrap 컨테이너를 찾습니다.
ranking_container = soup.find('div', class_='ranking-list-wrap')

if ranking_container:
    # ranking-item 클래스를 가진 모든 li 요소를 찾습니다.
    ranking_items = ranking_container.find_all('a', class_='ranking__info')

    for i, item in enumerate(ranking_items[:20]):
        # 순위를 가져오기 위해 i를 사용합니다.
        rank = i + 1

        # 각 항목에서 제목을 추출합니다.
        title = item.find('p', class_='info__title').text

        data.append(f"{rank} - {title}")

    # 데이터 출력
    for item in data:
        print(item)
else:
    print("컨테이너를 찾을 수 없습니다.")

ranking_container

<div class="ranking-list-wrap" data-v-b84bff86="" data-v-f5a64e4c=""><!-- --><!-- --><!-- --><!-- --></div>

In [5]:
!pip install PyQt5

In [6]:
!pip install PyQt5 PyQtWebEngine

- PyQt 상에서 웹 불러오기 및 크롤링 테스트

```python
import sys
import requests
from PyQt5.QtCore import QUrl
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QWidget, QPushButton
from PyQt5.QtWebEngineWidgets import QWebEngineView, QWebEnginePage
from bs4 import BeautifulSoup

class WebScraper(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setGeometry(100, 100, 800, 600)
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        layout = QVBoxLayout()

        self.web_view = QWebEngineView(self)
        self.web_page = QWebEnginePage()
        self.web_view.setPage(self.web_page)  # 웹 페이지 설정 추가
        self.web_view.setUrl(QUrl("https://m.kinolights.com/ranking/netflix"))  # 초기에 로드할 웹 페이지 설정
        layout.addWidget(self.web_view)

        self.button = QPushButton("클릭하여 사이트 로드", self)
        layout.addWidget(self.button)

        self.central_widget.setLayout(layout)

        self.button.clicked.connect(self.loadWebsite)

    def loadWebsite(self):
        url = QUrl("https://m.kinolights.com/ranking/netflix")  # 원하는 웹 페이지의 URL로 변경
        self.web_view.setUrl(url)

        # 웹 페이지 내용 가져오기
        page_content = self.web_page.toHtml(self.parseWebsite)

    def parseWebsite(self, html):
        # BeautifulSoup를 사용하여 HTML 내용 파싱
        soup = BeautifulSoup(html, 'html.parser')

        # data 리스트 초기화
        data = []

        # ranking-list-wrap 컨테이너를 찾습니다.
        ranking_container = soup.find('div', class_='ranking-list-wrap')

        if ranking_container:
            # ranking-item 클래스를 가진 모든 li 요소를 찾습니다.
            ranking_items = ranking_container.find_all('a', class_='ranking__info')

            for i, item in enumerate(ranking_items[:20]):
                # 순위를 가져오기 위해 i를 사용합니다.
                rank = i + 1

                # 각 항목에서 제목을 추출합니다.
                title = item.find('p', class_='info__title').text

                data.append(f"{rank} - {title}")

            # 데이터 출력
            for item in data:
                print(item)
        else:
            print("컨테이너를 찾을 수 없습니다.")

def main():
    app = QApplication(sys.argv)
    window = WebScraper()
    window.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()
```